<a href="https://colab.research.google.com/github/MikeXL/brain/blob/master/iris_torch.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!pip install torch_nightly -f https://download.pytorch.org/whl/nightly/cu90/torch_nightly.html

Looking in links: https://download.pytorch.org/whl/nightly/cu90/torch_nightly.html


In [0]:
import torch
import torch.nn as nn
from torch.autograd import Variable

import numpy as np
import pandas as pd

import matplotlib
import matplotlib.pyplot as plt

In [0]:
model = nn.Sequential(
  nn.Linear(13,24),
  nn.ReLU(),
  nn.Linear(24,12),
  nn.ReLU(),
  nn.Linear(12,3),
  nn.Softmax(1)
)

In [0]:
lf = nn.CrossEntropyLoss()

In [189]:
from sklearn import datasets
wine = datasets.load_wine()
wine.feature_names

['alcohol',
 'malic_acid',
 'ash',
 'alcalinity_of_ash',
 'magnesium',
 'total_phenols',
 'flavanoids',
 'nonflavanoid_phenols',
 'proanthocyanins',
 'color_intensity',
 'hue',
 'od280/od315_of_diluted_wines',
 'proline']

**!!** *normalize* the input

In [0]:
def train_lbfgs(model, Y, X, lf, epoch=200, lr=.01):
    loss = [0] * epoch
    optim = torch.optim.LBFGS(model.parameters(), lr=lr)
    for _ in range(epoch):
      def closure():
        yhat = model(X)
        l = lf(yhat, Y)
        loss[_] = l.item()
        optim.zero_grad()
        l.backward()
        return l
      optim.step(closure)
    return loss


In [0]:

model.train()
l = train_lbfgs(model, torch.from_numpy(wine.target).long(), torch.from_numpy(wine.data).float(), lf, 300, .01)   

In [0]:
plt.plot(l)

In [0]:
optim = torch.optim.Adam(model.parameters(), lr=.1)


def train(model, Y, X, optim, lf, epoch=200, lr=.1):
  loss = [0] * epoch
  for _ in range(epoch):
    yhat = model(X)
    l = lf(yhat, Y)
    loss[_] = l.item()
    optim.zero_grad()
    l.backward()
    optim.step()
  return loss


model.train()
l = train(model, torch.from_numpy(wine.target).long(), torch.from_numpy(wine.data).float(), optim, lf, 300)   

In [0]:
plt.plot(l)

In [0]:
with torch.no_grad():
  pred = model(torch.from_numpy(wine.data).float())
  l = lf(pred, torch.from_numpy(wine.target).long())
  
l

In [0]:
model.eval()
out=model(torch.from_numpy(wine.data).float())
_, p = torch.max(out, 1)

In [284]:
p

tensor([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
        1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2, 2,
        2, 2, 2, 2, 2, 2, 2, 2, 2, 2])

In [290]:
optim.state_dict

<bound method Adam.state_dict of Adam (
Parameter Group 0
    amsgrad: False
    betas: (0.9, 0.999)
    eps: 1e-08
    lr: 0.1
    weight_decay: 0
)>